<a href="https://colab.research.google.com/github/ahana02/AskYouTube/blob/main/Langchain_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Use GPU runtime

In [ ]:
pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.1 MB/s eta 0:00:00


In [ ]:
import whisper
import pytube

In [ ]:
# Reading the taken Youtube link
url = "https://www.youtube.com/watch?v=QDX-1M5Nj7s"
video = pytube.YouTube(url,use_oauth=False,
        allow_oauth_cache=True)
video.streams.get_highest_resolution().filesize

93449491

In [ ]:
# Converting and downloading as 'MP4' file
audio = video.streams.get_audio_only()
fn = audio.download(output_path="tmp.mp3")

In [ ]:
# Using the 'base' model of Whisper
model = whisper.load_model("base")

In [ ]:
transcription = model.transcribe("/content/tmp.mp3/MIT Introduction to Deep Learning  6S191.mp4")

In [ ]:
res = transcription['segments']
print(res)

In [ ]:
res[2]

{'id': 2,
 'seek': 3000,
 'start': 31.0,
 'end': 37.0,
 'text': ' This is a course about justice and we begin with a story.',
 'tokens': [50414,
  639,
  307,
  257,
  1164,
  466,
  6118,
  293,
  321,
  1841,
  365,
  257,
  1657,
  13,
  50714],
 'temperature': 0.0,
 'avg_logprob': -0.13037579329972415,
 'compression_ratio': 1.663716814159292,
 'no_speech_prob': 0.6196123957633972}

In [ ]:
from datetime import datetime

def store_segments(segments):
  texts = []
  start_times = []

  for segment in segments:
    text = segment['text']
    start = segment['start']

    # Convert the starting time to a datetime object
    start_datetime = datetime.fromtimestamp(start)

    # Format the starting time as a string in the format "00:00:00"
    formatted_start_time = start_datetime.strftime('%H:%M:%S')

    texts.append("".join(text))
    start_times.append(formatted_start_time)

  return texts, start_times

In [ ]:
store_segments(res)

([' Good afternoon, everyone. Thank you all for joining today. My name is Alexander Amini,',
  " and I'll be one of your course organizers this year, along with Ava. And together, we're",
  ' super excited to introduce you all to Introduction to Deep Learning. Now, MIT Inter to Deep Learning',
  ' is a really, really fun, exciting and fast-paced program here at MIT. And let me start by just',
  " first of all, giving you a little bit of background into what we do and what you're going to",
  " learn about this year. So this week of Introduction to Deep Learning, we're going to cover a ton",
  " of material in just one week. You'll learn the foundations of this really, really fascinating",
  " and exciting field of deep learning and artificial intelligence. And more importantly, you're",
  ' going to get hands-on experience actually reinforcing what you learn in the lectures as part of',
  ' hands-on software labs. Now, over the past decade, AI and deep learning have really had a huge',

In [ ]:
texts, start_times = store_segments(res)

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [ ]:
pip install faiss-gpu

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.chains import VectorDBQAWithSourcesChain
from langchain import OpenAI
import openai
import faiss

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1500, separator="\n")
docs = []
metadatas = []
for i, d in enumerate(texts):
    splits = text_splitter.split_text(d)
    docs.extend(splits)
    metadatas.extend([{"source": start_times[i]}] * len(splits))
embeddings = OpenAIEmbeddings()

In [ ]:
store = FAISS.from_texts(docs, embeddings, metadatas=metadatas)
faiss.write_index(store.index, "docs.index")

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
chain = VectorDBQAWithSourcesChain.from_llm(llm=OpenAI(temperature=0.9), vectorstore=store)

/usr/local/lib/python3.10/dist-packages/langchain/chains/qa_with_sources/vector_db.py:67: UserWarning: `VectorDBQAWithSourcesChain` is deprecated - please use `from langchain.chains import RetrievalQAWithSourcesChain`
  warnings.warn(


In [ ]:
result = chain({"question": "What is Backpropagation?"})

In [ ]:
print(f"Answer: {result['answer']}  Sources: {result['sources']}")

Answer:  Backpropagation is an algorithm used for training neural networks by propagating errors from the output back to the input.
  Sources: 00:42:28, 00:43:02, 00:43:21
